In [1]:
import os
import github as gh
import bs4 as bs
import requests
from lxml import etree

import selenium
from selenium import webdriver

from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager

import time
from tqdm import tqdm

import re

In [2]:
GITHUB_ACCESS_TOKEN=os.getenv('GITHUB_ACCESS_TOKEN')
g=gh.Github(GITHUB_ACCESS_TOKEN)
print(g.get_user())

AuthenticatedUser(login=None)


In [3]:
query='tensorflow language:python created:2021-04-01..2021-04-02'
urls = []
result=g.search_repositories(query)
for repository in result:
    urls.append(repository.html_url)
# print(result.totalCount)
# print(dir(result))
    

In [4]:
url='https://github.com/Tyler-Shamsuddoha/python-image-classifier-keras'
url2='https://github.com/bamblebam/image-classification-rps'

In [5]:
options=Options()
options.headless=True
driver=webdriver.Chrome(ChromeDriverManager(path='./').install(),options=options)
driver.get(url2)



====== WebDriver manager ======
Current google-chrome version is 91.0.4472
Get LATEST driver version for 91.0.4472
Get LATEST driver version for 91.0.4472
Trying to download new driver from https://chromedriver.storage.googleapis.com/91.0.4472.101/chromedriver_win32.zip
Driver has been saved in cache [./\drivers\chromedriver\win32\91.0.4472.101]


In [110]:
table=driver.find_element_by_xpath("//*[@class='Details-content--hidden-not-important js-navigation-container js-active-navigation-container d-md-block']")

In [111]:
# links=table.find_elements_by_tag_name('a')
# print(len(links))
# for link in links:
#     href=link.get_attribute('href')
#     if 'tree' in href or '.py' in href:
#         stack.append(href)

# driver.get(url2)
# table=driver.find_element_by_xpath("//*[@class='Details-content--hidden-not-important js-navigation-container js-active-navigation-container d-md-block']")
# links=table.find_elements_by_tag_name('a')
# for link in links:
#     href=link.get_attribute('href')
#     if '/tree/' in href or '.py' in href:
#         stack.append(href)

In [263]:
queue=list()
full_list=list()
links_list = set()
sequential_list=list()

In [7]:
def push_to_queue(links):
    for link in links:
        href=link.get_attribute('href')
        if href in links_list:
            continue
        if '/tree/' in href or '.py' in href:
            links_list.add(href)
            queue.append(href)

In [114]:
# def if_sequential():
#     code_body=driver.find_element_by_xpath("//*[@class='Box-body p-0 blob-wrapper data type-python  gist-border-0']")
#     if 'Sequential' in code_body.text:
#             count+=1
#     return count

In [22]:
def search_through_files(link):
    print(link)
    if '/tree/' in link and not 'venv' in link:
        driver.get(link)
        time.sleep(2.5)
        try:
            table=driver.find_element_by_xpath("//*[@class='Details-content--hidden-not-important js-navigation-container js-active-navigation-container d-block']")
            links=table.find_elements_by_tag_name('a')
            push_to_queue(links)
        except:
            print("No element found")
    elif '.py' in link and not 'venv' in link:
        driver.get(link)
        time.sleep(2.5)
        full_list.append(link)
        try: 
            code_body=driver.find_element_by_xpath("//*[@class='highlight tab-size js-file-line-container']")
            if 'Sequential' in code_body.text:
                sequential_list.append(link)
        except:
            print('Wrong class selector') 

In [9]:
def bfs():
    while queue:
        link=queue.pop(0)
        search_through_files(link)

In [10]:
def get_all_relevant_links(url):
    links_list.add(url)
    driver.get(url)
    try:
        table=driver.find_element_by_xpath("//*[@class='Details-content--hidden-not-important js-navigation-container js-active-navigation-container d-md-block']")
        links=table.find_elements_by_tag_name('a')
        for link in links:
            href=link.get_attribute('href')
            if ('/tree/' in href or '.py' in href) and not 'venv' in href:
                links_list.add(href)
                queue.append(href)
    except:
        print('No element found')
    bfs()
    print("Ended")

In [25]:
get_all_relevant_links(url)

https://github.com/Tyler-Shamsuddoha/python-image-classifier-keras/blob/master/preprocess.py
https://github.com/Tyler-Shamsuddoha/python-image-classifier-keras/blob/master/test.py
https://github.com/Tyler-Shamsuddoha/python-image-classifier-keras/blob/master/train.py
Ended


In [26]:
print(sequential_list)

['https://github.com/Tyler-Shamsuddoha/python-image-classifier-keras/blob/master/train.py']


In [76]:
driver.get('https://github.com/Tyler-Shamsuddoha/python-image-classifier-keras/blob/master/train.py')
# driver.get('https://github.com/fadhil-code/TensorFlowProject3/blob/master/Dropout.py')
time.sleep(2.5)
code_body=driver.find_element_by_xpath("//*[@class='Box-body p-0 blob-wrapper data type-python  gist-border-0']")

In [259]:

def getLayerSequence1helper(code):
    '''
    This is the function for models of type Sequential([...])
    :param code: the code to check layer sequence
    :return: array in correct sequence of layers
    :rtype: array model.add(.*)
    '''
    # re.findall('Sequential\((.*)',code,re.DOTALL)
    # re.findall('(Sequential\()(.+)((?:\n.+)+)(\]\))',code)
    # re.findall('(?s)_\(Sequential(.*?)\)',code) \[([^]]+)\]
    rawLayerSequence = re.findall('Sequential\(\[([^]]+)\]\)',code,re.DOTALL)
    layerSequence = list()
    temp=list()
    for i in rawLayerSequence:
        temp = re.findall('.?(.*)\)',i)
        layerSequence.append(temp)
    return layerSequence
    


In [248]:
def getCleanedLayers(layerlist):
    cleaned_layers=list()
    for model in layerlist:
        temp=list()
        for layer in model:
           word=layer.split('(')[0]
           word=''.join([char for char in word if char.isalnum()])
           temp.append(word)
        cleaned_layers.append(temp)
    return cleaned_layers

In [251]:
def getLayerSequence1(code):
    layerlist=getLayerSequence1helper(code)
    cleaned_layers=getCleanedLayers(layerlist)
    return cleaned_layers

In [11]:

def getLayerSequence2(code):
    '''
    This is the function for models of type model.add(layerName)
    :param code: the code to check layer sequence
    :return: array in correct sequence of layers
    :rtype: array 
    '''
    rawLayerSequence = re.findall('model.add(.*)',code)
    layerSequence = []
    for i in rawLayerSequence:
        temp = re.findall('keras.layers.?(.*)\(',i)
        layerSequence.append(temp[0])
    return layerSequence
    


In [266]:
def getSequentialModel(link):
    layerSequence = []
    get_all_relevant_links(link)
    for url in full_list:
        driver.get(url)
        time.sleep(2.5)
        print(url)
        try:
            code_body=driver.find_element_by_xpath("//*[@class='Box-body p-0 blob-wrapper data type-python  gist-border-0']")
            if 'Sequential' in code_body.text:
                layers1=getLayerSequence1(code_body.text)
                layers2 = getLayerSequence2(code_body.text)
                layerSequence.extend(layers1)
                layerSequence.append(layers2)
        except e:
            print(e)
    return layerSequence

In [267]:
testUrl = 'https://github.com/fadhil-code/TensorFlowProject3'
count = getSequentialModel(url)

https://github.com/Tyler-Shamsuddoha/python-image-classifier-keras/blob/master/preprocess.py
https://github.com/Tyler-Shamsuddoha/python-image-classifier-keras/blob/master/test.py
https://github.com/Tyler-Shamsuddoha/python-image-classifier-keras/blob/master/train.py
Ended
https://github.com/Tyler-Shamsuddoha/python-image-classifier-keras/blob/master/preprocess.py
https://github.com/Tyler-Shamsuddoha/python-image-classifier-keras/blob/master/test.py
https://github.com/Tyler-Shamsuddoha/python-image-classifier-keras/blob/master/train.py
https://github.com/Tyler-Shamsuddoha/python-image-classifier-keras/blob/master/preprocess.py
https://github.com/Tyler-Shamsuddoha/python-image-classifier-keras/blob/master/test.py
https://github.com/Tyler-Shamsuddoha/python-image-classifier-keras/blob/master/train.py


In [268]:
print(count)

[['Conv2D', 'MaxPooling2D', 'Conv2D', 'MaxPooling2D', 'Conv2D', 'MaxPooling2D', 'Conv2D', 'Flatten', 'Dense', 'Dropout', 'Dense'], ['Flatten', 'Dense', 'Dense'], [], ['Conv2D', 'MaxPooling2D', 'Conv2D', 'MaxPooling2D', 'Conv2D', 'MaxPooling2D', 'Conv2D', 'Flatten', 'Dense', 'Dropout', 'Dense'], ['Flatten', 'Dense', 'Dense'], []]


In [269]:
for i in count:
    print(i)

['Conv2D', 'MaxPooling2D', 'Conv2D', 'MaxPooling2D', 'Conv2D', 'MaxPooling2D', 'Conv2D', 'Flatten', 'Dense', 'Dropout', 'Dense']
['Flatten', 'Dense', 'Dense']
[]
['Conv2D', 'MaxPooling2D', 'Conv2D', 'MaxPooling2D', 'Conv2D', 'MaxPooling2D', 'Conv2D', 'Flatten', 'Dense', 'Dropout', 'Dense']
['Flatten', 'Dense', 'Dense']
[]
